# Installing Dependencies

In [2]:
pip install lyricsgenius

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 529.9 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from striprtf.striprtf import rtf_to_text
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import json
import lyricsgenius

# Reading Lyrics (.rtf) File and Converting to String

In [8]:
with open("When I Was Your Man.rtf") as file:
    content = file.read()
    text = rtf_to_text(content)
print(text)

"When I Was Your Man"

Same bed but it feels just a little bit bigger now
Our song on the radio but it don't sound the same
When our friends talk about you, all it does is just tear me down
'Cause my heart breaks a little when I hear your name

It all just sounds like oooooh…
Mmm, too young, too dumb to realize
That I should've bought you flowers
And held your hand
Should've gave you all my hours
When I had the chance
Take you to every party
'Cause all you wanted to do was dance
Now my baby's dancing
But she's dancing with another man

My pride, my ego, my needs, and my selfish ways
Caused a good strong woman like you to walk out my life
Now I'll never, never get to clean up the mess I made, ohh…
And it haunts me every time I close my eyes

It all just sounds like oooooh…
Mmm, too young, too dumb to realize
That I should've bought you flowers
And held your hand
Should've gave you all my hours
When I had the chance
Take you to every party
'Cause all you wanted to do was dance
Now my bab

# Using Genius API to Import Lyrics (main resource: https://github.com/johnwmillr/LyricsGenius#usage)

In [2]:
# Setting up access to API using access token from genius developer website
access_token = "_LIE8rLIMMSaGjV1B9lJ0PnI_coyEp0XeIclqImkRlN4JMtJ5OQPouOQ_RC5zf2i"
genius = lyricsgenius.Genius(access_token)

In [15]:
album = genius.search_album("Unorthodox Jukebox", "Bruno Mars")
album.save_lyrics()
# a .json file is saved in the directory containing information for the whole album

Searching for "Unorthodox Jukebox" by Bruno Mars...
Wrote Lyrics_UnorthodoxJukebox.json.


In [3]:
with open("Lyrics_UnorthodoxJukebox.json") as f:
    lyrics = json.load(f)

In [11]:
# lyrics now stores the .json file as a python string
print(lyrics['tracks'][3])

{'number': 4, 'song': {'_type': 'song', 'annotation_count': 5, 'api_path': '/songs/113496', 'artist_names': 'Bruno Mars', 'full_title': 'Treasure by\xa0Bruno\xa0Mars', 'header_image_thumbnail_url': 'https://images.genius.com/a011391dce77d483f6cfcb3b39a70b96.300x300x1.jpg', 'header_image_url': 'https://images.genius.com/a011391dce77d483f6cfcb3b39a70b96.1000x1000x1.jpg', 'id': 113496, 'instrumental': False, 'lyrics_owner_id': 181848, 'lyrics_state': 'complete', 'lyrics_updated_at': 1690485289, 'path': '/Bruno-mars-treasure-lyrics', 'pyongs_count': 72, 'relationships_index_url': 'https://genius.com/Bruno-mars-treasure-sample', 'release_date_components': {'year': 2012, 'month': 12, 'day': 7}, 'release_date_for_display': 'December 7, 2012', 'release_date_with_abbreviated_month_for_display': 'Dec. 7, 2012', 'song_art_image_thumbnail_url': 'https://images.genius.com/c7b37e0ffbe3b7143883c795475e7bcc.300x300x1.jpg', 'song_art_image_url': 'https://images.genius.com/c7b37e0ffbe3b7143883c795475e7b

In [67]:
song_lyrics_dict = {"song": [], "lyrics": []}
for track in lyrics['tracks']:
    new_lyrics = track['song']['lyrics']
    start_index = new_lyrics.find('[Verse 1]')
    new_lyrics = new_lyrics[start_index:]
    
    end_index = new_lyrics.find('Embed')
    new_lyrics = new_lyrics[:(end_index-2)]
    
    song_lyrics_dict["song"].append(track['song']['title_with_featured'])
    song_lyrics_dict["lyrics"].append(new_lyrics)

In [71]:
print(song_lyrics_dict["song"][4])
print("-------")
print(song_lyrics_dict["lyrics"][4])

Moonshine
-------
[Verse 1]
Hello
You know, you look even better than the way you did the night before
And the moment that you kiss my lips, you know, I start to feel wonderful
It's somethin' incredible
There's sex in your chemicals, oh
Ooh, let's go
You're the best way I know to escape the extraordinary
This world ain't for you and I know for damn sure this world ain't for me
Lift off and say goodbye
Just let your fire set me free

[Chorus]
Oh, moonshine
Take us to the stars tonight
Take us to that special place
That place we went the last time, the last time

[Verse 2]
I know
I was with you last night but it feels like it's been so long
And everybody that's around, they notice that I'm not myself when you're gone
It's good to see you again
Good to see you again
On top of the world (Uh)
Is where I stand when you're back in my life
Life's not so bad when you're way up this high
Everything is alright
Everything is alright
See Bruno Mars LiveGet tickets as low as $97You might also like[C

# Sentiment Analyzer Model